In [1]:
import os

In [2]:
os.chdir('..')

<img src="flow_4.png">

In [3]:
from flows.flows import Flows

Welcome to the Data Science Package. First create an object as follows:
For example, use the code below to import the flow 0:
flow = Flows(0)
You can define the `categorical_threshold` which is the maximum number of categories that a categorical feature should have before considering it as continuous numeric feature. The default value is 50
For example, use the code below to import the flow 0 with defining the categorical_threshold as 50
flow = Flows(flow_id=0, categorical_threshold=50)


In [4]:
flow = Flows(4)

Please use the following function to read the data
dataframe_dict, columns_set = flow.load_data(path : str, files_list : list)
For example: path = './data'
If your data is in a nested directory, it is better to os.path.join. For example: path = os.path.join('data', 'flow_0')
For example: files_list = ['train.csv','test.csv']
The output is a dictionary that contains dataframes e.g.  
dataframe_dict = {'train': train_dataframe,'test': test_dataframe}
If you want to explore the data you can run one of the following functions: 
1 . flow.exploring_data(dataframe_dict: dict, key_i: str)
For example: flow.exploring_data(dataframe_dict, 'train')
2 . flow.comparing_statistics(dataframe_dict: dict)
For example: flow.comparing_statistics(dataframe_dict)




In [5]:
path = "./data/flow_3" # the flow applied to the same data in flow_3
files_list = ['train_transaction.csv','test_transaction.csv']

In [6]:
dataframe_dict, columns_set = flow.load_data(path, files_list)

A summary of the data sets


,train_transaction,test_transaction
column type,,
categorical_integer,204,190
categorical_string,14,14
continuous,176,189
date,0,0
json,0,0
other,0,0
total amount,394,393


NOTE: numeric categorical columns that contains more than 50 classes are considered numeric continuous features.
NOTE: You can modify the threshold value if you want to consider more or less numeric categorical features as numeric continuous features.
The possible ids are {'TransactionID'}
The possible possible_target are ['isFraud']
The type of the problem that should be solved {'isFraud': 'classification'}

The Kullback-Leibler Divergence between probability mass function (pmf) derived from `isFraud` and and an uniformly distributed pmf 	 = 1.001 
It is above the threshold 	 0.050. 
Imballanced target variable!


,total count,frequency (%)
isFraud values,,
0,569877,96.5
1,20663,3.5


If you have categorical features with string labels, Encode the categorical features by applying the following function:
dataframe_dict, columns_set = flow.encode_categorical_feature(dataframe_dict: dict)


In [7]:
dataframe_dict["train_transaction"].head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
 columns_set["train_transaction"].keys()

dict_keys(['categorical_string', 'categorical_integer', 'continuous', 'date', 'json', 'other'])

In [9]:
dataframe_dict, columns_set = flow.encode_categorical_feature(dataframe_dict,print_results=10)

The reference dataframe is: train_transaction
there are 2 datasets provided
encoding the feature in the dataset test_transaction
the number of classes in ProductCD feature is: 5
there are 2 datasets provided
encoding the feature in the dataset test_transaction
the number of classes in card4 feature is: 5
there are 2 datasets provided
encoding the feature in the dataset test_transaction
the number of classes in card6 feature is: 5
there are 2 datasets provided
encoding the feature in the dataset test_transaction
the number of classes in P_emaildomain feature is: 61
there are 2 datasets provided
encoding the feature in the dataset test_transaction
the number of classes in R_emaildomain feature is: 61
there are 2 datasets provided
encoding the feature in the dataset test_transaction
the number of classes in M1 feature is: 3
there are 2 datasets provided
encoding the feature in the dataset test_transaction
the number of classes in M2 feature is: 3
there are 2 datasets provided
encoding the

,train_transaction,test_transaction,all data sets
string columns,,,
isFraud,2,NaN,2
ProductCD,5,5.0,5
card4,5,5.0,5
card6,5,4.0,5
P_emaildomain,60,61.0,61
R_emaildomain,61,61.0,61
C3,27,32.0,33
D9,25,25.0,25
M1,3,3.0,3


******************************
A summary of the data sets


,train_transaction,test_transaction
column type,,
categorical_integer,214,201
categorical_string,0,0
continuous,180,192
date,0,0
json,0,0
other,0,0
total amount,394,393


NOTE: numeric categorical columns that contains more than 50 classes are considered numeric continuous features.
NOTE: You can modify the threshold value if you want to consider more or less numeric categorical features as numeric continuous features.
If some features have in all rows the same value, they have no influence on the target prediction. It is a good idea to delete such features:
dataframe_dict, columns_set = flow.drop_columns_constant_values(dataframe_dict: dict, ignore_columns: list)An example of the ignore_columns list: 
 ignore_columns = [target]



In [10]:
ignore_columns = ['isFraud']

In [11]:
dataframe_dict, columns_set = flow.drop_columns_constant_values(dataframe_dict, ignore_columns)

Constant columns count: 0
394 columns total
394 columns left
The set of remaining columns should be modified. Error: "['isFraud'] not in index"
A summary of the data sets


,train_transaction,test_transaction
column type,,
categorical_integer,214,201
categorical_string,0,0
continuous,180,192
date,0,0
json,0,0
other,0,0
total amount,394,393


NOTE: numeric categorical columns that contains more than 50 classes are considered numeric continuous features.
NOTE: You can modify the threshold value if you want to consider more or less numeric categorical features as numeric continuous features.
If some features are highly correlated, they do not provide more information about the target prediction. It is a good idea to drop such features but keep one:
dataframe_dict, columns_set = flow.drop_correlated_columns(dataframe_dict: dict, ignore_columns: list)An example of the ignore_columns list: 
 ignore_columns = [target]



In [12]:
dataframe_dict, columns_set = flow.drop_correlated_columns(dataframe_dict, ignore_columns)

Columns with correlations more than 0.98 :
TransactionID-->TransactionDT: r^2=0.9982802236567436
TransactionDT-->TransactionID: r^2=0.9982802236567436
C1-->C2: r^2=0.9950894591945199
C1-->C6: r^2=0.9822377721825558
C1-->C11: r^2=0.9965147332620219
C2-->C1: r^2=0.9950894591945199
C6-->C14: r^2=0.9842010703541241
C7-->C8: r^2=0.9829833764568252
C7-->C10: r^2=0.9850616676486351
C7-->C12: r^2=0.9994888253007479
C8-->C7: r^2=0.9829833764568252
D1-->D2: r^2=0.9813113910391011
D2-->D1: r^2=0.9813113910391011
D4-->D12: r^2=0.9999994444505867
D5-->D7: r^2=0.9864959630082745
D7-->D5: r^2=0.9864959630082745
D12-->D4: r^2=0.9999994444505867
V1-->V2: r^2=0.9965407936468617
V1-->V3: r^2=0.9941701044518824
V1-->V4: r^2=0.9963688310819988
V1-->V5: r^2=0.9959836480520691
V1-->V6: r^2=0.9933627660410614
V1-->V7: r^2=0.9896492485207462
V1-->V8: r^2=0.9978313634150651
V1-->V9: r^2=0.9968006920487128
V1-->V10: r^2=0.9907001474532228
V1-->V11: r^2=0.9942943741409519
V2-->V1: r^2=0.9965407936468617
V12-->V13

148 columns left
The set of remaining columns should be modified. Error: "['isFraud'] not in index"
A summary of the data sets


,train_transaction,test_transaction
column type,,
categorical_integer,49,50
categorical_string,0,0
continuous,99,97
date,0,0
json,0,0
other,0,0
total amount,148,147


NOTE: numeric categorical columns that contains more than 50 classes are considered numeric continuous features.
NOTE: You can modify the threshold value if you want to consider more or less numeric categorical features as numeric continuous features.
Apply target-based encoding to the categorical features by applying the following function:

dataframe_dict, columns_set = flow.features_encoding("target", dataframe_dict: dict, reference: str, ignore_columns: list, target: str)


In [13]:
ignore_columns = ["TransactionID", "isFraud"]
reference = "train_transaction"
target = "isFraud"

In [14]:
dataframe_dict, columns_set = flow.features_encoding("target",
                                                     dataframe_dict,
                                                     reference=reference,
                                                     ignore_columns=ignore_columns,
                                                     target_name=target)

A summary of the data sets


,train_transaction,test_transaction
column type,,
categorical_integer,49,50
categorical_string,0,0
continuous,99,97
date,0,0
json,0,0
other,0,0
total amount,148,147


NOTE: numeric categorical columns that contains more than 50 classes are considered numeric continuous features.
NOTE: You can modify the threshold value if you want to consider more or less numeric categorical features as numeric continuous features.
If you have numeric features, it is a good idea to normalize numeric features.Use the following function for feature normalization:
 dataframe_dict, columns_set = flow.scale_data (dataframe_dict: dict, ignore_columns: list)
For example: ignore_columns = ['id', 'target']


In [15]:
ignore_columns = ["TransactionID", "isFraud"]

In [16]:
dataframe_dict, columns_set = flow.scale_data(dataframe_dict, ignore_columns)

A summary of the data sets


,train_transaction,test_transaction
column type,,
categorical_integer,49,48
categorical_string,0,0
continuous,99,99
date,0,0
json,0,0
other,0,0
total amount,148,147


NOTE: numeric categorical columns that contains more than 50 classes are considered numeric continuous features.
NOTE: You can modify the threshold value if you want to consider more or less numeric categorical features as numeric continuous features.
If you want to explore the data you can run one of the following functions: 
1 . flow.exploring_data(dataframe_dict: dict, key_i: str)
For example: flow.exploring_data(dataframe_dict, 'train')
2 . flow.comparing_statistics(dataframe_dict: dict)
For example: flow.comparing_statistics(dataframe_dict)


Your features are ready to train the model: 
You can start training the model by applying the following function: 
model_index_list, save_models_dir, y_test = flow.training(parameters)
parameters = {
     "data": {
         "train": {"features": train_dataframe, "target": train_target}, 
         "valid": {"features": valid_dataframe, "target": valid_target}, 
         "test": {"features": test_dataframe, "target": test_target}, 
     }, 
   

In [17]:
flow.exploring_data(dataframe_dict, "test_transaction")

interactive(children=(IntSlider(value=0, description='column_i', max=146), Output()), _dom_classes=('widget-in…

In [18]:
flow.comparing_statistics(dataframe_dict)

interactive(children=(IntSlider(value=0, description='feature_nr', max=147), Output()), _dom_classes=('widget-…

In [23]:
columns = dataframe_dict["train_transaction"].columns
total_columns = columns_set["train_transaction"]["continuous"] +\
columns_set["train_transaction"]["categorical_integer"]

train_dataframe = dataframe_dict["train_transaction"][
    [x for x in total_columns if x not in ignore_columns]]
test_dataframe = dataframe_dict["test_transaction"][
    [x for x in total_columns if x not in ignore_columns]]
train_target = dataframe_dict["train_transaction"]["isFraud"]

In [20]:
parameters = {
    "data": {
        "train": {"features": train_dataframe, "target": train_target.to_numpy()},
    },
    "split": {
        "method": "kfold",  # "method":"kfold"
        "fold_nr": 5,  # foldnr:5 , "split_ratios": 0.8 # "split_ratios":(0.7,0.2)
    },
    "model": {"type": "Logistic regression",
              "hyperparameters": {"alpha": "optimize",  # alpha:optimize
                                  },
              },
    "metrics": ["accuracy_score", "roc_auc_score"],
    "predict": {
        "test": {"features": test_dataframe}
    }
}

In [21]:
model_index_list_logistic, save_models_dir_logistic, y_test_logistic = flow.training(parameters)

shuffle is not provided: 'shuffle'
random_state is not provided: 'random_state'
fold_nr. 1
{'accuracy_score (train.train)': 0.9695, 'roc_auc_score (train.train)': 0.5853691893104174, 'accuracy_score (train.validation)': 0.9694, 'roc_auc_score (train.validation)': 0.5835156523913411}
fold_nr. 2
{'accuracy_score (train.train)': 0.9693, 'roc_auc_score (train.train)': 0.5849530665917194, 'accuracy_score (train.validation)': 0.9701, 'roc_auc_score (train.validation)': 0.5853410256459373}
fold_nr. 3


KeyboardInterrupt: 

In [24]:
parameters_lighgbm = {
    "data": {
        "train": {"features": train_dataframe, "target": train_target.to_numpy()},
    },
    "split": {
        "method": "kfold",  # "method":"kfold"
        "fold_nr": 5,  # foldnr:5 , "split_ratios": 0.8 # "split_ratios":(0.7,0.2)
    },
    "model": {"type": "lightgbm",
              "hyperparameters": dict(objective='binary', metric='cross-entropy', num_leaves=5,
                                      boost_from_average=True,
                                      learning_rate=0.05, bagging_fraction=0.99, feature_fraction=0.99, max_depth=-1,
                                      num_rounds=10000, min_data_in_leaf=10, boosting='dart')
              },
    "metrics": ["accuracy_score", "roc_auc_score"],
    "predict": {
        "test": {"features": test_dataframe}
    }
}

In [ ]:
model_index_list_lightgbm, save_models_dir_lightgbm, y_test_lightgbm = flow.training(parameters_lighgbm)

shuffle is not provided: 'shuffle'
random_state is not provided: 'random_state'
fold_nr. 1
{'accuracy_score (train.train)': 0.9785, 'roc_auc_score (train.train)': 0.9353048062330105, 'accuracy_score (train.validation)': 0.9777, 'roc_auc_score (train.validation)': 0.9248609663044632}
fold_nr. 2


In [ ]:
parameters_xgboost = {
    "data": {
        "train": {"features": train_dataframe, "target": train_target.to_numpy()},
    },
    "split": {
        "method": "kfold",  # "method":"kfold"
        "fold_nr": 5,  # fold_nr:5 , "split_ratios": 0.3 # "split_ratios":(0.3,0.2)
    },
    "model": {"type": "xgboost",
              "hyperparameters": {'max_depth': 5, 'eta': 1, 'eval_metric': "auc", 'objective':'binary:logistic'}
              },
    "metrics": ["accuracy_score", "roc_auc_score"],
    "predict": {
        "test": {"features": test_dataframe}
    }
}

In [ ]:
model_index_list_xgboost, save_models_dir_xgboost, y_test_xgboost = flow.training(parameters_xgboost)

In [ ]:
train_dataframe = dataframe_dict['train_transaction'].drop(['isFraud'],1)
train_target = dataframe_dict["train_transaction"]["isFraud"]

test_dataframe = dataframe_dict['test_transaction']  #.drop(['isFraud'],1)

parameters_sklearn = {
    "data": {
        "train": {
            "features": train_dataframe,
            "target": train_target.to_numpy()
        }, 
    },
    "split": {
        "method": "split",
        "split_ratios": 0.2,
        "stratify": True
    },
    "model": {
        "type": "sklearn.ensemble.RandomForestClassifier",
        "hyperparameters": {
            'params_grid':{
                'class_weight': ["balanced_subsample"],
                'max_depth': [5, 10, 20, 999],
                'min_samples_leaf': [10, 3, 1]
            },
            'params_fixed': {
                'max_depth': 4, 
                'min_samples_split': 10, 
                'min_samples_leaf': 2, 
                'random_state': 11
            },
            'params_cv': {
                'n_splits': 5, 
                'shuffle': True, 
                'random_state': 11
            },
            "objective": "classification",
            "grid_search_scoring": ['roc_auc', 'accuracy']
        },
    },
    "metrics": ['roc_auc_score', 'accuracy_score'],
    "predict": {
        "test": {
            "features": test_dataframe
        }
    }
}

In [ ]:
model_index_list_RF, save_models_dir_RF, y_test_RF = flow.training(parameters_sklearn)